In [1]:
# !pip install colab-env --upgrade

In [2]:
# !pip install --upgrade --quiet neptune-client


In [3]:
# !pip install geopandas

In [4]:
# !pip install plotly-express

In [5]:
# !pip install plotly -U

In [6]:
# from google.colab import drive
# drive.mount('/content/drive')

In [7]:
# import colab_env

In [8]:
# colab_env.__version__

In [9]:
import os
from dotenv import load_dotenv
load_dotenv("/content/drive/MyDrive/Geospatial-article/Notebooks/vars.env")

#Neptune API token
API_token = os.getenv("API_token")

#Mapbox credentials
style_url = os.getenv("style_url")
access_token = os.getenv("access_token")

In [10]:
import neptune.new as neptune
run = neptune.init(project='codebrain/Geospatial-article',
                   api_token=API_token) # your credentials

Info (NVML): Driver Not Loaded. GPU usage metrics may not be reported. For more information, see https://docs-legacy.neptune.ai/logging-and-managing-experiment-results/logging-experiment-data.html#hardware-consumption 


https://app.neptune.ai/codebrain/Geospatial-article/e/GEOS-2


In [11]:
# importing libraries
import pandas as pd
pd.set_option('display.max_columns', None)
import geopandas as gpd
import plotly.express as px

In [12]:
#reading datasets
health_df = gpd.read_file('/content/drive/MyDrive/Geospatial-article/Datasets/health-care.geojson')
adminstrative_df = gpd.read_file('/content/drive/MyDrive/Geospatial-article/Datasets/lga.geojson')
pop_df = gpd.read_file('/content/drive/MyDrive/Geospatial-article/Datasets/NGA_population.json')

In [13]:
health_df.columns

Index(['id', 'latitude', 'longitude', 'contact_phone', 'contact_name',
       'ri_service_status', 'timestamp', 'cce_quantity', 'cce_available',
       'cce_lastupdated', 'category', 'global_id', 'name', 'alternate_name',
       'functional_status', 'type', 'ownership', 'ward_code', 'accessibility',
       'source', 'ward_name', 'lga_code', 'lga_name', 'state_code',
       'state_name', 'geometry'],
      dtype='object')

In [14]:
adminstrative_df.columns

Index(['id', 'geometry_type', 'amap_code', 'source', 'lga_code', 'lga_name',
       'state_code', 'global_id', 'state_name', 'geometry'],
      dtype='object')

In [15]:
pop_df.columns

Index(['lgacode', 'lganame', 'statecode', 'source', 'timestamp', 'globalid',
       'amapcode', 'id', 'statename', 'region', 'mean', 'q025', 'q05', 'q25',
       'q50', 'q75', 'q95', 'q975', 'geometry'],
      dtype='object')

# # Cleaning the datasets

In [16]:
health_df = health_df[
    ['latitude', 'longitude','functional_status','type', 'lga_name','state_name', 'geometry']]

adminstrative_df = adminstrative_df[['lga_name','state_name','geometry']]
pop_df = pop_df[['lganame','mean', 'statename','geometry']]

In [17]:
pop_df = pop_df.replace(to_replace=['Ajeromi Ifelodun', 'Ifako Ijaye','Oshodi Isolo' ],
                       value=['Ajeromi/Ifelodun','Ifako/Ijaye', 'Oshodi/Isolo']).reset_index()
pop_df = pop_df.rename(columns={'lganame':'lga_name'})
pop_df.drop('index', axis=1, inplace=True)

# Data Engineering

In [18]:
health_pop = health_df.merge(pop_df, how='left', on='lga_name')
health_pop.drop(columns=['statename','geometry_x'],axis=1, inplace=True)
health_pop.rename(columns={'geometry_y':'geometry'}, inplace=True)

total_hospital_count = health_pop.groupby('lga_name')['geometry'].count().reset_index()
hosp_per_x = total_hospital_count.merge(pop_df, how='left', on='lga_name')
hosp_per_x.rename(columns={'geometry_x':'count', 'mean':'mean_pop', 'geometry_y':'geometry'}, inplace=True)
hosp_per_x['Health_facilities_per_100000'] = (hosp_per_x['count']/hosp_per_x['mean_pop'])*100000
hosp_per_x.drop(columns=['count','mean_pop', 'statename'], axis=1, inplace=True)
hosp_per_100000 = hosp_per_x[['lga_name','Health_facilities_per_100000']]

hosp_type_df = health_df.replace(to_replace=['Primary', 'Secondary','Tertiary' ],
                       value=[1,2,3]).reset_index()
hosp_type_df.drop('index', axis=1, inplace=True)

# Scatterplot

In [19]:
fig1 = px.scatter_mapbox(health_df, lat="latitude", lon="longitude", color="functional_status", hover_data=["type", "lga_name"],
                        zoom=8, height=300,
                       labels={'functional_status':'Functional status of Health Facilities'},
                       center={'lat': 6.5355, 'lon': 3.3087}
)
fig1.update_layout(mapbox_style=style_url, mapbox_accesstoken=access_token)
fig1.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
run['interactive__scatter_plot_img'] = neptune.types.File.as_html(fig1)
fig1.show()

# Chloropleth Map

In [20]:
import json 
  
# Opening JSON file 
f = open('/content/drive/MyDrive/Geospatial-article/Datasets/lga.geojson',) 
  
# returns JSON object as  a dictionary 
geo_json_file = json.load(f) 

In [21]:
fig2 = px.choropleth_mapbox(hosp_per_100000, 
                     geojson=geo_json_file, 
                     locations='lga_name', 
                     color='Health_facilities_per_100000', 
                     featureidkey="properties.lga_name",  
                     range_color=(0, 100),
                     labels={'Health_facilities_per_100000':'Health Facilities_per_100000'},
                     zoom=8.5,
                     center={'lat': 6.5355, 'lon': 3.3087}
                          )
fig2.update_layout(mapbox_style=style_url, mapbox_accesstoken=access_token)

fig2.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
run['interactive__chloropleth_map_img'] = neptune.types.File.as_html(fig2)
fig2.show()

# Heatmap

In [22]:
hosp_type_df

,latitude,longitude,functional_status,type,lga_name,state_name,geometry
0,6.624977,3.319815,Functional,1,Agege,Lagos,POINT (3.31980 6.62500)
1,6.619202,3.320412,Functional,1,Agege,Lagos,POINT (3.32040 6.61920)
2,6.612740,3.313860,Unknown,1,Agege,Lagos,POINT (3.31390 6.61270)
3,6.617422,3.316163,Functional,1,Agege,Lagos,POINT (3.31620 6.61740)
4,6.622920,3.318610,Functional,1,Agege,Lagos,POINT (3.31860 6.62290)
...,...,...,...,...,...,...,...
1582,6.457886,3.394287,Unknown,1,Lagos Island,Lagos,POINT (3.39430 6.45790)
1583,6.454980,3.392070,Functional,1,Lagos Island,Lagos,POINT (3.39210 6.45500)
1584,6.455688,3.392273,Unknown,1,Lagos Island,Lagos,POINT (3.39230 6.45570)
1585,6.453674,3.393311,Unknown,1,Lagos Island,Lagos,POINT (3.39330 6.45370)


In [23]:
fig3 = px.density_mapbox(hosp_type_df, lat='latitude', lon='longitude', z='type', radius=10,
                        center={'lat': 6.5355, 'lon': 3.3087}, zoom=8.5,
                        labels={'type':'Health Facilities type'},
                        )
fig3.update_layout(mapbox_style=style_url, mapbox_accesstoken=access_token)
fig3.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
run['interactive__heatmap_map_img'] = neptune.types.File.as_html(fig3)
fig3.show()

# Line on map

In [26]:
import plotly.graph_objects as go
fig4 = go.Figure(go.Scattermapbox(
    mode = "markers+lines",
    lat = [6.5095,6.6018, 6.4698],
    lon = [3.3711,3.3515, 3.5852],
    marker = {'size': 10}))
fig4.update_layout(mapbox_style=style_url, mapbox_accesstoken=access_token)
fig4.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig4.update_layout(mapbox= dict(
    center={'lat': 6.5355, 'lon': 3.3087},zoom=8.5 ))
run['interactive__line_on_map_img'] = neptune.types.File.as_html(fig4)
fig4.show()

# Filled Area

In [27]:
fig5 = go.Figure(go.Scattermapbox(
    fill = "toself",
    lon = [3.297806, 3.295470, 3.349685, 3.346413], lat = [6.539536,6.488922, 6.488922, 6.542322],
    marker = { 'size': 10, 'color': "red" }))
fig5.update_layout(mapbox_style=style_url, mapbox_accesstoken=access_token)
fig5.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig5.update_layout(mapbox= dict(
    center={'lat': 6.5355, 'lon': 3.3087},zoom=10))
run['interactive__filled_area_img'] = neptune.types.File.as_html(fig5)
fig5.show()